In [1]:
import numpy as np
import pandas as pd

In [2]:
######     Edit: Edit sheet name    ######  
df= pd.read_excel('bb_report.xlsx',sheet_name='ranking')

In [3]:
######     Ranking summary        ######  
df1=df
ranking_summary = df1.groupby(['assigned_month', 'assigned_week'])["total_shipments", "rank_1","rank_2","rank_3"].apply(lambda x : x.astype(int).sum())
ranking_summary['Ranking percentage']= round((ranking_summary['rank_1']+ranking_summary['rank_2']+ranking_summary['rank_3'])*100/ranking_summary['total_shipments'])
# for i in ranking_summary.index:
#     ranking_summary.at[i,'Ranking percentage'] = round((ranking_summary.at[i,'rank_1']+ranking_summary.at[i,'rank_2']+ranking_summary.at[i,'rank_3'])*100/ranking_summary.at[i,'total_shipments'])
ranking_summary = ranking_summary.reset_index()
ranking_summary = ranking_summary[ranking_summary.assigned_week != "Total"]
ranking_summary.drop(['total_shipments', 'rank_1','rank_2','rank_3'], axis='columns', inplace=True)
ranking_summary = ranking_summary.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
ranking_summary=ranking_summary.sort_values(by='Month', ascending=True)
ranking_summary["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
ranking_summary

<ipython-input-3-924ee06da9c2>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ranking_summary = df1.groupby(['assigned_month', 'assigned_week'])["total_shipments", "rank_1","rank_2","rank_3"].apply(lambda x : x.astype(int).sum())


,Month,Week,Ranking percentage
1,September,week4,97.0
3,October,week1,94.0
4,October,week2,89.0
5,October,week3,89.0
6,October,week4,84.0
8,November,week1,88.0
9,November,week2,94.0
10,November,week3,93.0
11,November,week4,89.0
13,December,week1,86.0


In [4]:
######     Get company names      ######  
cols = [0, 1, 4]
df_comp_names = pd.read_excel('sr_company_summary_last_12_months.xlsx',sheet_name='shipments', usecols=cols)

In [5]:
######   Ranking CID      ######  
ranking_cid = df1.groupby(['assigned_month', 'assigned_week','company_id'])["total_shipments", "rank_1","rank_2","rank_3"].apply(lambda x : x.astype(int).sum())
ranking_cid['Ranking percentage']=round((ranking_cid['rank_1']+ranking_cid['rank_2']+ranking_cid['rank_3'])*100/ranking_cid['total_shipments'])
# for i in ranking_cid.indexranking_cid
#     ranking_cid.at[i,'Ranking percentage'] = round((ranking_cid.at[i,'rank_1']+ranking_cid.at[i,'rank_2']+ranking_cid.at[i,'rank_3'])*100/ranking_cid.at[i,'total_shipments'])
ranking_cid = ranking_cid.reset_index()
ranking_cid = ranking_cid[ranking_cid.assigned_week != "Total"]
# ranking_cid_numbers=ranking_cid[['company_id','total_shipments', 'rank_1','rank_2','rank_3']]
# ranking_cid.drop(['total_shipments', 'rank_1','rank_2','rank_3'], axis='columns', inplace=True)
ranking_cid = ranking_cid.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
ranking_cid["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
ranking_cid_final = ranking_cid.pivot_table(['Ranking percentage','total_shipments'], ['company_id'],['Month', 'Week'])
ranking_cid_final = ranking_cid_final.reset_index()
ranking_cid_final = ranking_cid_final.sort_values('company_id')
ranking_cid_final['company_id'] = ranking_cid_final['company_id'].astype(int)
ranking_cid_final = ranking_cid_final.sort_values('company_id')

#Assign company names
myset= set(ranking_cid['company_id'])
cids=list(myset)
df_comp_names=df_comp_names.loc[df_comp_names['company_id'].isin(cids)]
df_comp_names = df_comp_names.sort_values('company_id')
df_comp_names['company_name'] = df_comp_names['company_name'].astype(str)
df_comp_names['tier'] = df_comp_names['tier'].astype(str)
cnames = df_comp_names['company_name'].tolist()
ctier = df_comp_names['tier'].tolist()
ranking_cid_final.insert(0, 'Tier', ctier)
ranking_cid_final.insert(0, 'Company Name', cnames)
# ranking_cid_final['Company Name']=cnames
# ranking_cid_final['Tier']=ctier
ranking_cid_final

<ipython-input-5-2777e5b009c1>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ranking_cid = df1.groupby(['assigned_month', 'assigned_week','company_id'])["total_shipments", "rank_1","rank_2","rank_3"].apply(lambda x : x.astype(int).sum())


Company Name    Tier company_id Ranking percentage        \
Month                                                       December         
Week                                                           week1 week2   
216              Tanvi Herbals  SILVER        809               94.0  91.0   
63                  Mistiq Ark  BRONZE       2311                NaN   NaN   
74                   GRANDSELL  BRONZE       3023                NaN   NaN   
195                elizona.com  BRONZE       7389                NaN   NaN   
218                 SLATE HASH  BRONZE       8094                NaN   NaN   
..                         ...     ...        ...                ...   ...   
7      Bismillah K.G.N Express  BRONZE    1027117                NaN   NaN   
8             The_tree nursery  BRONZE    1039630                NaN   NaN   
9                Ganapati Bags  BRONZE    1047038                NaN   NaN   
10             Genius Infotech  BRONZE    1049176                NaN   NaN   
11                Bro Ministry  BRONZE    1052324                NaN   NaN   

                                           ... total_shipments           \
Month              January November        ...         January November   
Week  week3  week4   week1    week1 week2  ...           week1    week1   
216    86.0   86.0     NaN     90.0  87.0  ...             NaN    187.0   
63      NaN  100.0     NaN      NaN   NaN  ...             NaN      NaN   
74      NaN  100.0   100.0      NaN   NaN  ...             5.0      NaN   
195     NaN  100.0   100.0      NaN   NaN  ...             1.0      NaN   
218     NaN  100.0     NaN      NaN   NaN  ...             NaN      NaN   
..      ...    ...     ...      ...   ...  ...             ...      ...   
7       NaN  100.0     NaN      NaN   NaN  ...             NaN      NaN   
8       NaN  100.0   100.0      NaN   NaN  ...             6.0      NaN   
9       NaN  100.0     NaN      NaN   NaN  ...             NaN      NaN   
10      NaN  100.0   100.0      NaN   NaN  ...             1.0      NaN   
11      NaN   79.0     NaN      NaN   NaN  ...             NaN      NaN   

                                                                   
Month                      October                      September  
Week   week2  week3  week4   week1  week2  week3  week4     week4  
216    159.0  148.0  239.0   187.0  192.0  143.0  213.0     176.0  
63       NaN    NaN    NaN     NaN    NaN    NaN    NaN       NaN  
74       NaN    NaN    NaN     NaN    NaN    NaN    NaN       NaN  
195      NaN    NaN    NaN     NaN    NaN    NaN    NaN       NaN  
218      NaN    NaN    NaN     NaN    NaN    NaN    NaN       NaN  
..       ...    ...    ...     ...    ...    ...    ...       ...  
7        NaN    NaN    NaN     NaN    NaN    NaN    NaN       NaN  
8        NaN    NaN    NaN     NaN    NaN    NaN    NaN       NaN  
9        NaN    NaN    NaN     NaN    NaN    NaN    NaN       NaN  
10       NaN    NaN    NaN     NaN    NaN    NaN    NaN       NaN  
11       NaN    NaN    NaN     NaN    NaN    NaN    NaN       NaN  

[254 rows x 31 columns]

In [6]:
######     Edit:Ranking CID order       ######  
ranking_cid_final = ranking_cid_final.reset_index()
# ranking_cid_final = ranking_cid_final.set_index('Company Name')
ranking_cid_final = ranking_cid_final.set_index(['Company Name', 'Tier','company_id'])
# ranking_cid_final.drop(['index'], axis='columns', inplace=True)
cols_order = ['September', 'October','November','December','January','February','March','April','May','June','July','August']
ranking_cid_final = ranking_cid_final.reindex(cols_order, axis=1, level=1)
ranking_cid_final

Ranking percentage                \
Month                                              September October         
Week                                                   week4   week1 week2   
Company Name            Tier   company_id                                    
Tanvi Herbals           SILVER 809                      89.0    94.0  92.0   
Mistiq Ark              BRONZE 2311                      NaN     NaN   NaN   
GRANDSELL               BRONZE 3023                      NaN     NaN   NaN   
elizona.com             BRONZE 7389                      NaN     NaN   NaN   
SLATE HASH              BRONZE 8094                      NaN     NaN   NaN   
...                                                      ...     ...   ...   
Bismillah K.G.N Express BRONZE 1027117                   NaN     NaN   NaN   
The_tree nursery        BRONZE 1039630                   NaN     NaN   NaN   
Ganapati Bags           BRONZE 1047038                   NaN     NaN   NaN   
Genius Infotech         BRONZE 1049176                   NaN     NaN   NaN   
Bro Ministry            BRONZE 1052324                   NaN     NaN   NaN   

                                                                            \
Month                                                 November               
Week                                      week3 week4    week1 week2 week3   
Company Name            Tier   company_id                                    
Tanvi Herbals           SILVER 809         96.0  90.0     90.0  87.0  91.0   
Mistiq Ark              BRONZE 2311         NaN   NaN      NaN   NaN   NaN   
GRANDSELL               BRONZE 3023         NaN   NaN      NaN   NaN   NaN   
elizona.com             BRONZE 7389         NaN   NaN      NaN   NaN   NaN   
SLATE HASH              BRONZE 8094         NaN   NaN      NaN   NaN   NaN   
...                                         ...   ...      ...   ...   ...   
Bismillah K.G.N Express BRONZE 1027117      NaN   NaN      NaN   NaN   NaN   
The_tree nursery        BRONZE 1039630      NaN   NaN      NaN   NaN   NaN   
Ganapati Bags           BRONZE 1047038      NaN   NaN      NaN   NaN   NaN   
Genius Infotech         BRONZE 1049176      NaN   NaN      NaN   NaN   NaN   
Bro Ministry            BRONZE 1052324      NaN   NaN      NaN   NaN   NaN   

                                                          ... total_shipments  \
Month                                           December  ...         October   
Week                                      week4    week1  ...           week4   
Company Name            Tier   company_id                 ...                   
Tanvi Herbals           SILVER 809         93.0     94.0  ...           213.0   
Mistiq Ark              BRONZE 2311         NaN      NaN  ...             NaN   
GRANDSELL               BRONZE 3023         NaN      NaN  ...             NaN   
elizona.com             BRONZE 7389         NaN      NaN  ...             NaN   
SLATE HASH              BRONZE 8094         NaN      NaN  ...             NaN   
...                                         ...      ...  ...             ...   
Bismillah K.G.N Express BRONZE 1027117      NaN      NaN  ...             NaN   
The_tree nursery        BRONZE 1039630      NaN      NaN  ...             NaN   
Ganapati Bags           BRONZE 1047038      NaN      NaN  ...             NaN   
Genius Infotech         BRONZE 1049176      NaN      NaN  ...             NaN   
Bro Ministry            BRONZE 1052324      NaN      NaN  ...             NaN   

                                                                         \
Month                                     November                        
Week                                         week1  week2  week3  week4   
Company Name            Tier   company_id                                 
Tanvi Herbals           SILVER 809           187.0  159.0  148.0  239.0   
Mistiq Ark              BRONZE 2311            NaN    NaN    NaN    NaN   
GRANDSELL               BRONZE 302

In [7]:
df_pickup= pd.read_excel('bb_report.xlsx',sheet_name='pickup')
df1_pickup=df_pickup

In [8]:
pickup_summary = df1_pickup.groupby(['assigned_month', 'assigned_week'])["total_shipments", "pickup_not_generated","not_picked","gt_2_day",'0_day', '1_day', '2_day'].apply(lambda x : x.astype(int).sum())
pickup_summary = pickup_summary.reset_index()
pickup_summary = pickup_summary[pickup_summary.assigned_week != "Total"]
pickup_summary['Pickup Generated'] = pickup_summary['total_shipments'] - pickup_summary['pickup_not_generated']
pickup_summary['Pick < 2 days'] = pickup_summary['0_day']+pickup_summary['1_day']+pickup_summary['2_day']
pickup_summary['Not Picked (Percentage)'] = round(pickup_summary['not_picked']*100/pickup_summary['Pickup Generated'])
pickup_summary['Picked in < 2 days (Percentage)'] = round(pickup_summary['Pick < 2 days']*100/pickup_summary['Pickup Generated'])
pickup_summary.drop(['1_day', '0_day','2_day','Pick < 2 days'], axis='columns', inplace=True)
pickup_summary = pickup_summary.rename(columns={"assigned_month": "Month", "assigned_week": "Week","gt_2_day":"Picked in > 2 days"})
# pickup_summary["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December"}, inplace=True)
pickup_summary=pickup_summary.sort_values(by='Month', ascending=True)
pickup_summary["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
cols = pickup_summary.columns.tolist()
cols_final = ['Month',
 'Week',
 'total_shipments',
 'pickup_not_generated',
 'Pickup Generated',
 'not_picked', 
 'Not Picked (Percentage)',
 'Picked in > 2 days',
 'Picked in < 2 days (Percentage)']
pickup_summary=pickup_summary[cols_final]
pickup_summary

<ipython-input-8-dc289fe66d71>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pickup_summary = df1_pickup.groupby(['assigned_month', 'assigned_week'])["total_shipments", "pickup_not_generated","not_picked","gt_2_day",'0_day', '1_day', '2_day'].apply(lambda x : x.astype(int).sum())


,Month,Week,total_shipments,pickup_not_generated,Pickup Generated,not_picked,Not Picked (Percentage),Picked in > 2 days,Picked in < 2 days (Percentage)
1,September,week4,6253,0,6253,0,0.0,138,98.0
3,October,week1,5388,0,5388,0,0.0,255,95.0
4,October,week2,4694,0,4694,0,0.0,125,97.0
5,October,week3,4798,0,4798,0,0.0,504,89.0
6,October,week4,6105,1,6104,0,0.0,514,92.0
8,November,week1,4555,0,4555,0,0.0,166,96.0
9,November,week2,3935,0,3935,0,0.0,222,94.0
10,November,week3,2734,0,2734,0,0.0,107,96.0
11,November,week4,2741,0,2741,0,0.0,87,97.0
13,December,week1,1849,0,1849,0,0.0,117,94.0


In [9]:
pickup_cid = df1_pickup.groupby(['assigned_month', 'assigned_week','company_id'])["total_shipments","pickup_not_generated","not_picked","gt_2_day",'0_day', '1_day', '2_day'].apply(lambda x : x.astype(int).sum())

<ipython-input-9-32634753c66b>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pickup_cid = df1_pickup.groupby(['assigned_month', 'assigned_week','company_id'])["total_shipments","pickup_not_generated","not_picked","gt_2_day",'0_day', '1_day', '2_day'].apply(lambda x : x.astype(int).sum())


In [10]:
pickup_cid['Pickup Generated'] = pickup_cid['total_shipments'] - pickup_cid['pickup_not_generated']
pickup_cid['Pick < 2 days'] = pickup_cid['0_day']+pickup_cid['1_day']+pickup_cid['2_day']
pickup_cid['Not Picked (Percentage)'] = round(pickup_cid['not_picked']*100/pickup_cid['Pickup Generated'])
pickup_cid['Picked in < 2 days (Percentage)'] = round(pickup_cid['Pick < 2 days']*100/pickup_cid['Pickup Generated'])
pickup_cid.drop(['1_day', '0_day','2_day','Pick < 2 days'], axis='columns', inplace=True)
pickup_cid = pickup_cid.reset_index()
pickup_cid = pickup_cid.rename(columns={"assigned_month": "Month", "assigned_week": "Week","gt_2_day":"Picked in > 2 days"})
pickup_cid["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
pickup_cid = pickup_cid[pickup_cid.Week != "Total"]
pickup_cid_final = pickup_cid.pivot_table(['total_shipments',
 'pickup_not_generated',
 'not_picked',
 'Picked in > 2 days',
 'Pickup Generated',
 'Not Picked (Percentage)',
 'Picked in < 2 days (Percentage)'], ['company_id'],['Month', 'Week'])
pickup_cid_final = pickup_cid_final.reset_index()
pickup_cid_final = pickup_cid_final.sort_values('company_id')
pickup_cid_final['company_id'] = pickup_cid_final['company_id'].astype(int)
pickup_cid_final = pickup_cid_final.sort_values('company_id')
# pickup_cid_final['Company Name']=cnames
# pickup_cid_final['Tier']=ctier
pickup_cid_final.insert(0, 'Tier', ctier)
pickup_cid_final.insert(0, 'Company Name', cnames)

In [11]:
pickup_cid_final = pickup_cid_final.reset_index()
# pickup_cid_final = pickup_cid_final.set_index('Company Name')
pickup_cid_final = pickup_cid_final.set_index(['Company Name', 'Tier','company_id'])
# pickup_cid_final.drop(['index'], axis='columns', inplace=True)
pickup_cid_final = pickup_cid_final.reindex(cols_order, axis=1, level=1)
pickup_cid_final

Not Picked (Percentage)          \
Month                                                   September October   
Week                                                        week4   week1   
Company Name            Tier   company_id                                   
Tanvi Herbals           SILVER 809                            0.0     0.0   
Mistiq Ark              BRONZE 2311                           NaN     NaN   
GRANDSELL               BRONZE 3023                           NaN     NaN   
elizona.com             BRONZE 7389                           NaN     NaN   
SLATE HASH              BRONZE 8094                           NaN     NaN   
...                                                           ...     ...   
Bismillah K.G.N Express BRONZE 1027117                        NaN     NaN   
The_tree nursery        BRONZE 1039630                        NaN     NaN   
Ganapati Bags           BRONZE 1047038                        NaN     NaN   
Genius Infotech         BRONZE 1049176                        NaN     NaN   
Bro Ministry            BRONZE 1052324                        NaN     NaN   

                                                                            \
Month                                                       November         
Week                                      week2 week3 week4    week1 week2   
Company Name            Tier   company_id                                    
Tanvi Herbals           SILVER 809          0.0   0.0   0.0      0.0   0.0   
Mistiq Ark              BRONZE 2311         NaN   NaN   NaN      NaN   NaN   
GRANDSELL               BRONZE 3023         NaN   NaN   NaN      NaN   NaN   
elizona.com             BRONZE 7389         NaN   NaN   NaN      NaN   NaN   
SLATE HASH              BRONZE 8094         NaN   NaN   NaN      NaN   NaN   
...                                         ...   ...   ...      ...   ...   
Bismillah K.G.N Express BRONZE 1027117      NaN   NaN   NaN      NaN   NaN   
The_tree nursery        BRONZE 1039630      NaN   NaN   NaN      NaN   NaN   
Ganapati Bags           BRONZE 1047038      NaN   NaN   NaN      NaN   NaN   
Genius Infotech         BRONZE 1049176      NaN   NaN   NaN      NaN   NaN   
Bro Ministry            BRONZE 1052324      NaN   NaN   NaN      NaN   NaN   

                                                                ...  \
Month                                                 December  ...   
Week                                      week3 week4    week1  ...   
Company Name            Tier   company_id                       ...   
Tanvi Herbals           SILVER 809          0.0   0.0      0.0  ...   
Mistiq Ark              BRONZE 2311         NaN   NaN      NaN  ...   
GRANDSELL               BRONZE 3023         NaN   NaN      NaN  ...   
elizona.com             BRONZE 7389         NaN   NaN      NaN  ...   
SLATE HASH              BRONZE 8094         NaN   NaN      NaN  ...   
...                                         ...   ...      ...  ...   
Bismillah K.G.N Express BRONZE 1027117      NaN   NaN      NaN  ...   
The_tree nursery        BRONZE 1039630      NaN   NaN      NaN  ...   
Ganapati Bags           BRONZE 1047038      NaN   NaN      NaN  ...   
Genius Infotech         BRONZE 1049176      NaN   NaN      NaN  ...   
Bro Ministry            BRONZE 1052324      NaN   NaN      NaN  ...   

                                          total_shipments                  \
Month                                             October November          
Week                                                week4    week1  week2   
Company Name            Tier   company_id                                   
Tanvi Herbals           SILVER 809                  213.0    187.0  159.0   
Mistiq Ark              BRONZE 2311                   NaN      NaN    NaN   
GRANDSELL               BRONZE 3023                   NaN      NaN    NaN   
elizona.com             BRONZE 7389                   NaN      NaN    NaN   
SLATE HASH              BRONZE 80

In [12]:
df_rto= pd.read_excel('bb_report.xlsx',sheet_name='rto')
df1_rto=df_rto
rto_summary = df1_rto.groupby(['assigned_month', 'assigned_week'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())
rto_summary['RTO percentage'] = round(rto_summary['rto_shipments']*100/rto_summary['total_shipments'])
rto_summary = rto_summary.reset_index()
rto_summary = rto_summary[rto_summary.assigned_week != "Total"]
rto_summary = rto_summary.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
rto_summary=rto_summary.sort_values(by='Month', ascending=True)
rto_summary["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
rto_summary


<ipython-input-12-f80e6d6151ad>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rto_summary = df1_rto.groupby(['assigned_month', 'assigned_week'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())


,Month,Week,total_shipments,rto_shipments,RTO percentage
1,September,week4,6253,1831,29.0
3,October,week1,5388,1386,26.0
4,October,week2,4694,1723,37.0
5,October,week3,4798,2025,42.0
6,October,week4,6105,3055,50.0
8,November,week1,4555,2139,47.0
9,November,week2,3935,1692,43.0
10,November,week3,2734,1078,39.0
11,November,week4,2741,958,35.0
13,December,week1,1849,872,47.0


In [13]:
df1_rto_pre = df1_rto[df1_rto.payment_mode != "cod"]
rto_prepaid_summary = df1_rto_pre.groupby(['assigned_month', 'assigned_week'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())
rto_prepaid_summary['RTO percentage'] = round(rto_prepaid_summary['rto_shipments']*100/rto_prepaid_summary['total_shipments'])
rto_prepaid_summary = rto_prepaid_summary.reset_index()
rto_prepaid_summary = rto_prepaid_summary[rto_prepaid_summary.assigned_week != "Total"]
rto_prepaid_summary = rto_prepaid_summary.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
rto_prepaid_summary=rto_prepaid_summary.sort_values(by='Month', ascending=True)
rto_prepaid_summary["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
rto_prepaid_summary

<ipython-input-13-eba0134e7674>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rto_prepaid_summary = df1_rto_pre.groupby(['assigned_month', 'assigned_week'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())


,Month,Week,total_shipments,rto_shipments,RTO percentage
1,September,week4,1593,20,1.0
3,October,week1,1959,19,1.0
4,October,week2,679,15,2.0
5,October,week3,727,5,1.0
6,October,week4,609,6,1.0
8,November,week1,683,8,1.0
9,November,week2,492,8,2.0
10,November,week3,278,2,1.0
11,November,week4,367,4,1.0
13,December,week1,229,0,0.0


In [14]:
df1_rto_cod = df1_rto[df1_rto.payment_mode != "prepaid"]
rto_cod_summary = df1_rto_cod.groupby(['assigned_month', 'assigned_week'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())
rto_cod_summary['RTO percentage'] = round(rto_cod_summary['rto_shipments']*100/rto_cod_summary['total_shipments'])
rto_cod_summary = rto_cod_summary.reset_index()
rto_cod_summary = rto_cod_summary[rto_cod_summary.assigned_week != "Total"]
rto_cod_summary = rto_cod_summary.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
rto_cod_summary=rto_cod_summary.sort_values(by='Month', ascending=True)
rto_cod_summary["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
rto_cod_summary

<ipython-input-14-f88e67f0648a>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rto_cod_summary = df1_rto_cod.groupby(['assigned_month', 'assigned_week'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())


,Month,Week,total_shipments,rto_shipments,RTO percentage
1,September,week4,4660,1811,39.0
3,October,week1,3429,1367,40.0
4,October,week2,4015,1708,43.0
5,October,week3,4071,2020,50.0
6,October,week4,5496,3049,55.0
8,November,week1,3872,2131,55.0
9,November,week2,3443,1684,49.0
10,November,week3,2456,1076,44.0
11,November,week4,2374,954,40.0
13,December,week1,1620,872,54.0


In [15]:
rto_cid = df1_rto.groupby(['assigned_month', 'assigned_week','company_id'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())
rto_cid['RTO percentage'] = round(rto_cid['rto_shipments']*100/rto_cid['total_shipments'])
rto_cid = rto_cid.reset_index()
rto_cid = rto_cid[rto_cid.assigned_week != "Total"]
rto_cid = rto_cid.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
rto_cid["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
rto_cid=rto_cid.sort_values(by='Month', ascending=False)
rto_cid = rto_cid.pivot_table(['RTO percentage','total_shipments'], ['company_id'],['Month', 'Week'])
rto_cid = rto_cid.reset_index()
rto_cid = rto_cid.sort_values('company_id')
rto_cid['company_id'] = rto_cid['company_id'].astype(int)
rto_cid = rto_cid.sort_values('company_id')
# rto_cid['Company Name']=cnames
# rto_cid['Tier']=ctier
rto_cid.insert(0, 'Tier', ctier)
rto_cid.insert(0, 'Company Name', cnames)
rto_cid = rto_cid.reset_index()
# rto_cid = rto_cid.set_index('Company Name')
rto_cid = rto_cid.set_index(['Company Name', 'Tier','company_id'])
# rto_cid.drop(['index'], axis='columns', inplace=True)
rto_cid = rto_cid.reindex(cols_order, axis=1, level=1)
rto_cid

<ipython-input-15-97d6242c4eab>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rto_cid = df1_rto.groupby(['assigned_month', 'assigned_week','company_id'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())


RTO percentage                      \
Month                                          September October               
Week                                               week4   week1 week2 week3   
Company Name            Tier   company_id                                      
Tanvi Herbals           SILVER 809                  14.0    16.0  17.0  14.0   
Mistiq Ark              BRONZE 2311                  NaN     NaN   NaN   NaN   
GRANDSELL               BRONZE 3023                  NaN     NaN   NaN   NaN   
elizona.com             BRONZE 7389                  NaN     NaN   NaN   NaN   
SLATE HASH              BRONZE 8094                  NaN     NaN   NaN   NaN   
...                                                  ...     ...   ...   ...   
Bismillah K.G.N Express BRONZE 1027117               NaN     NaN   NaN   NaN   
The_tree nursery        BRONZE 1039630               NaN     NaN   NaN   NaN   
Ganapati Bags           BRONZE 1047038               NaN     NaN   NaN   NaN   
Genius Infotech         BRONZE 1049176               NaN     NaN   NaN   NaN   
Bro Ministry            BRONZE 1052324               NaN     NaN   NaN   NaN   

                                                                            \
Month                                           November                     
Week                                      week4    week1 week2 week3 week4   
Company Name            Tier   company_id                                    
Tanvi Herbals           SILVER 809         16.0     16.0  15.0  14.0  13.0   
Mistiq Ark              BRONZE 2311         NaN      NaN   NaN   NaN   NaN   
GRANDSELL               BRONZE 3023         NaN      NaN   NaN   NaN   NaN   
elizona.com             BRONZE 7389         NaN      NaN   NaN   NaN   NaN   
SLATE HASH              BRONZE 8094         NaN      NaN   NaN   NaN   NaN   
...                                         ...      ...   ...   ...   ...   
Bismillah K.G.N Express BRONZE 1027117      NaN      NaN   NaN   NaN   NaN   
The_tree nursery        BRONZE 1039630      NaN      NaN   NaN   NaN   NaN   
Ganapati Bags           BRONZE 1047038      NaN      NaN   NaN   NaN   NaN   
Genius Infotech         BRONZE 1049176      NaN      NaN   NaN   NaN   NaN   
Bro Ministry            BRONZE 1052324      NaN      NaN   NaN   NaN   NaN   

                                                    ... total_shipments  \
Month                                     December  ...         October   
Week                                         week1  ...           week4   
Company Name            Tier   company_id           ...                   
Tanvi Herbals           SILVER 809            15.0  ...           213.0   
Mistiq Ark              BRONZE 2311            NaN  ...             NaN   
GRANDSELL               BRONZE 3023            NaN  ...             NaN   
elizona.com             BRONZE 7389            NaN  ...             NaN   
SLATE HASH              BRONZE 8094            NaN  ...             NaN   
...                                            ...  ...             ...   
Bismillah K.G.N Express BRONZE 1027117         NaN  ...             NaN   
The_tree nursery        BRONZE 1039630         NaN  ...             NaN   
Ganapati Bags           BRONZE 1047038         NaN  ...             NaN   
Genius Infotech         BRONZE 1049176         NaN  ...             NaN   
Bro Ministry            BRONZE 1052324         NaN  ...             NaN   

                                                                         \
Month                                     November                        
Week                                         week1  week2  week3  week4   
Company Name            Tier   company_id                                 
Tanvi Herbals           SILVER 809           187.0  159.0  148.0  239.0   
Mistiq Ark              BRONZE 2311            NaN    NaN    NaN    NaN   
GRANDSELL               BRONZE 3023            NaN    NaN    NaN    NaN   
elizona.com        

In [16]:
rto_cid_pre = df1_rto_pre.groupby(['assigned_month', 'assigned_week','company_id'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())
rto_cid_pre['RTO percentage'] = round(rto_cid_pre['rto_shipments']*100/rto_cid_pre['total_shipments'])
rto_cid_pre = rto_cid_pre.reset_index()
rto_cid_pre = rto_cid_pre[rto_cid_pre.assigned_week != "Total"]
rto_cid_pre = rto_cid_pre.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
rto_cid_pre["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
rto_cid_pre=rto_cid_pre.sort_values(by='Month', ascending=False)
rto_cid_pre = rto_cid_pre.pivot_table(['RTO percentage','total_shipments'], ['company_id'],['Month', 'Week'])
rto_cid_pre = rto_cid_pre.reset_index()
rto_cid_pre = rto_cid_pre.sort_values('company_id')
rto_cid_pre['company_id'] = rto_cid_pre['company_id'].astype(int)
rto_cid_pre = rto_cid_pre.sort_values('company_id')
# rto_cid_pre['Company Name']=cnames
# rto_cid_pre['Tier']=ctier
rto_cid_pre = rto_cid_pre.reset_index()
rto_cid_pre = rto_cid_pre.set_index('company_id')
rto_cid_pre.drop(['index'], axis='columns', inplace=True)
rto_cid_pre = rto_cid_pre.reindex(cols_order, axis=1, level=1)
rto_cid_pre

<ipython-input-16-6ab702161d8e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rto_cid_pre = df1_rto_pre.groupby(['assigned_month', 'assigned_week','company_id'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())
C:\Users\abc\anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


RTO percentage                                                 \
Month           September October                   November               
Week                week4   week1 week2 week3 week4    week1 week2 week3   
company_id                                                                 
809                   0.0     1.0   1.0   0.0   1.0      0.0   1.0   0.0   
2311                  NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
3023                  NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
7389                  NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
8094                  NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
...                   ...     ...   ...   ...   ...      ...   ...   ...   
1022128               NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
1027117               NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
1039630               NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
1047038               NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
1049176               NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   

                           ... total_shipments                              \
Month            December  ...         October November                      
Week       week4    week1  ...           week4    week1 week2 week3  week4   
company_id                 ...                                               
809          1.0      0.0  ...           118.0     89.0  72.0  73.0  122.0   
2311         NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
3023         NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
7389         NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
8094         NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
...          ...      ...  ...             ...      ...   ...   ...    ...   
1022128      NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
1027117      NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
1039630      NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
1047038      NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
1049176      NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   

                                               
Month      December                   January  
Week          week1 week2 week3 week4   week1  
company_id                                     
809            94.0  90.0  92.0  42.0     NaN  
2311            NaN   NaN   NaN   1.0     NaN  
3023            NaN   NaN   NaN   1.0     5.0  
7389            NaN   NaN   NaN   3.0     1.0  
8094            NaN   NaN   NaN   2.0     NaN  
...             ...   ...   ...   ...     ...  
1022128         NaN   NaN   NaN   2.0     1.0  
1027117         NaN   NaN   NaN  13.0     NaN  
1039630         NaN   NaN   NaN   4.0     6.0  
1047038         NaN   NaN   NaN   3.0     NaN  
1049176         NaN   NaN   NaN   1.0     1.0  

[216 rows x 28 columns]

In [17]:
rto_cid_cod = df1_rto_cod.groupby(['assigned_month', 'assigned_week','company_id'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())
rto_cid_cod['RTO percentage'] = round(rto_cid_cod['rto_shipments']*100/rto_cid_cod['total_shipments'])
rto_cid_cod = rto_cid_cod.reset_index()
rto_cid_cod = rto_cid_cod[rto_cid_cod.assigned_week != "Total"]
rto_cid_cod = rto_cid_cod.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
rto_cid_cod["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
rto_cid_cod=rto_cid_cod.sort_values(by='Month', ascending=False)
rto_cid_cod = rto_cid_cod.pivot_table(['RTO percentage','total_shipments'], ['company_id'],['Month', 'Week'])
rto_cid_cod = rto_cid_cod.reset_index()
rto_cid_cod = rto_cid_cod.sort_values('company_id')
rto_cid_cod['company_id'] = rto_cid_cod['company_id'].astype(int)
rto_cid_cod = rto_cid_cod.sort_values('company_id')
# rto_cid_cod['Company Name']=cnames
# rto_cid_cod['Tier']=ctier
rto_cid_cod = rto_cid_cod.reset_index()
rto_cid_cod = rto_cid_cod.set_index('company_id')
rto_cid_cod.drop(['index'], axis='columns', inplace=True)
rto_cid_cod = rto_cid_cod.reindex(cols_order, axis=1, level=1)
rto_cid_cod

<ipython-input-17-c1f27ca2332a>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rto_cid_cod = df1_rto_cod.groupby(['assigned_month', 'assigned_week','company_id'])["total_shipments", "rto_shipments"].apply(lambda x : x.astype(int).sum())
C:\Users\abc\anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


RTO percentage                                                 \
Month           September October                   November               
Week                week4   week1 week2 week3 week4    week1 week2 week3   
company_id                                                                 
809                  25.0    32.0  28.0  27.0  35.0     30.0  26.0  27.0   
7389                  NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
8880                  NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
11602                 NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
15382                 NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
...                   ...     ...   ...   ...   ...      ...   ...   ...   
1010615               NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
1016834               NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
1020207               NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
1022128               NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   
1052324               NaN     NaN   NaN   NaN   NaN      NaN   NaN   NaN   

                           ... total_shipments                              \
Month            December  ...         October November                      
Week       week4    week1  ...           week4    week1 week2 week3  week4   
company_id                 ...                                               
809         26.0     32.0  ...            95.0     98.0  87.0  75.0  117.0   
7389         NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
8880         NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
11602        NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
15382        NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
...          ...      ...  ...             ...      ...   ...   ...    ...   
1010615      NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
1016834      NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
1020207      NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
1022128      NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   
1052324      NaN      NaN  ...             NaN      NaN   NaN   NaN    NaN   

                                                
Month      December                    January  
Week          week1 week2 week3  week4   week1  
company_id                                      
809            88.0  85.0  72.0   45.0     NaN  
7389            NaN   NaN   NaN    1.0     NaN  
8880            NaN   NaN   NaN    7.0     1.0  
11602           NaN   NaN   NaN   71.0    10.0  
15382           NaN   NaN   NaN    4.0     NaN  
...             ...   ...   ...    ...     ...  
1010615         NaN   NaN   NaN   36.0     NaN  
1016834         NaN   NaN   NaN  432.0   248.0  
1020207         NaN   NaN   NaN    1.0     NaN  
1022128         NaN   NaN   NaN    3.0     2.0  
1052324         NaN   NaN   NaN   28.0     NaN  

[153 rows x 28 columns]

In [18]:
df_delivery= pd.read_excel('bb_report.xlsx',sheet_name='delivery')
df1_delivery=df_delivery
delivery_summary = df1_delivery.groupby(['assigned_month','assigned_week']).agg({'total_attempted_shipments':'sum', 
                                                                                     'first_attempt_deliver':'sum',
                                                                                     'no_of_shipments_within_edd':'sum',
                                                                                     'avg_first_attempt_time':'mean',
                                                                                     'avg_delivered_time':'mean'})
delivery_summary=delivery_summary.reset_index()
delivery_summary = delivery_summary[delivery_summary.assigned_week != "Total"]
delivery_summary[['avg_first_attempt_time','avg_delivered_time']]=round(delivery_summary[['avg_first_attempt_time','avg_delivered_time']])
delivery_summary['First_attempt_delivery (Percentage)'] = round(delivery_summary['first_attempt_deliver']*100/delivery_summary['total_attempted_shipments'])
delivery_summary['Percentage within EDD'] = round(delivery_summary['no_of_shipments_within_edd']*100/delivery_summary['total_attempted_shipments'])
delivery_summary = delivery_summary.reset_index()
delivery_summary = delivery_summary[delivery_summary.assigned_week != "Total"]
delivery_summary = delivery_summary.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
delivery_summary=delivery_summary.sort_values(by='Month', ascending=True)
delivery_summary["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
cols_final=[ 'Month',
 'Week',
 'total_attempted_shipments',
 'first_attempt_deliver',
 'First_attempt_delivery (Percentage)',
 'no_of_shipments_within_edd',
 'Percentage within EDD',
 'avg_first_attempt_time',
 'avg_delivered_time']
delivery_summary=delivery_summary[cols_final]
delivery_summary

,Month,Week,total_attempted_shipments,first_attempt_deliver,First_attempt_delivery (Percentage),no_of_shipments_within_edd,Percentage within EDD,avg_first_attempt_time,avg_delivered_time
0,September,week4,6220,3826,62.0,4933,79.0,90.0,95.0
1,October,week1,5356,3498,65.0,4786,89.0,91.0,98.0
2,October,week2,4664,2477,53.0,3865,83.0,97.0,108.0
3,October,week3,4778,2333,49.0,2178,46.0,91.0,101.0
4,October,week4,6075,2558,42.0,4626,76.0,94.0,102.0
5,November,week1,4531,2092,46.0,3413,75.0,88.0,96.0
6,November,week2,3907,1842,47.0,2256,58.0,107.0,115.0
7,November,week3,2723,1384,51.0,2066,76.0,86.0,90.0
8,November,week4,2725,1479,54.0,1915,70.0,88.0,92.0
9,December,week1,1825,791,43.0,1215,67.0,86.0,93.0


In [19]:
delivery_cid = df1_delivery.groupby(['assigned_month','assigned_week','company_id']).agg({'total_attempted_shipments':'sum', 
                                                                                     'first_attempt_deliver':'sum',
                                                                                     'no_of_shipments_within_edd':'sum',
                                                                                     'avg_first_attempt_time':'mean',
                                                                                     'avg_delivered_time':'mean'})
delivery_cid=delivery_cid.reset_index()
delivery_cid = delivery_cid[delivery_cid.assigned_week != "Total"]
delivery_cid[['avg_first_attempt_time','avg_delivered_time']]=round(delivery_cid[['avg_first_attempt_time','avg_delivered_time']])
delivery_cid['First_attempt_delivery (Percentage)'] = round(delivery_cid['first_attempt_deliver']*100/delivery_cid['total_attempted_shipments'])
delivery_cid['Percentage within EDD'] = round(delivery_cid['no_of_shipments_within_edd']*100/delivery_cid['total_attempted_shipments'])
delivery_cid = delivery_cid.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
delivery_cid=delivery_cid.sort_values(by='Month', ascending=True)
delivery_cid["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
delivery_cid_final = delivery_cid.pivot_table([ 'Percentage within EDD',
                                               'First_attempt_delivery (Percentage)',
 'total_attempted_shipments',
 'first_attempt_deliver',
 'no_of_shipments_within_edd',
 'avg_first_attempt_time',
 'avg_delivered_time'], ['company_id'],['Month', 'Week'])
delivery_cid_final = delivery_cid_final.reset_index()
delivery_cid_final = delivery_cid_final.sort_values('company_id')
delivery_cid_final['company_id'] = delivery_cid_final['company_id'].astype(int)
delivery_cid_final = delivery_cid_final.sort_values('company_id')
# delivery_cid_final['Company Name']=cnames
# delivery_cid_final['Tier']=ctier
delivery_cid_final.insert(0, 'Tier', ctier)
delivery_cid_final.insert(0, 'Company Name', cnames)

In [20]:
delivery_cid_final = delivery_cid_final.reset_index()
# delivery_cid_final = delivery_cid_final.set_index('Company Name')
delivery_cid_final = delivery_cid_final.set_index(['Company Name', 'Tier','company_id'])
# delivery_cid_final.drop(['index'], axis='columns', inplace=True)
delivery_cid_final = delivery_cid_final.reindex(cols_order, axis=1, level=1)
delivery_cid_final

First_attempt_delivery (Percentage)  \
Month                                                               September   
Week                                                                    week4   
Company Name            Tier   company_id                                       
Tanvi Herbals           SILVER 809                                       78.0   
Mistiq Ark              BRONZE 2311                                       NaN   
GRANDSELL               BRONZE 3023                                       NaN   
elizona.com             BRONZE 7389                                       NaN   
SLATE HASH              BRONZE 8094                                       NaN   
...                                                                       ...   
Bismillah K.G.N Express BRONZE 1027117                                    NaN   
The_tree nursery        BRONZE 1039630                                    NaN   
Ganapati Bags           BRONZE 1047038                                    NaN   
Genius Infotech         BRONZE 1049176                                    NaN   
Bro Ministry            BRONZE 1052324                                    NaN   

                                                                              \
Month                                     October                   November   
Week                                        week1 week2 week3 week4    week1   
Company Name            Tier   company_id                                      
Tanvi Herbals           SILVER 809           66.0  71.0  78.0  75.0     75.0   
Mistiq Ark              BRONZE 2311           NaN   NaN   NaN   NaN      NaN   
GRANDSELL               BRONZE 3023           NaN   NaN   NaN   NaN      NaN   
elizona.com             BRONZE 7389           NaN   NaN   NaN   NaN      NaN   
SLATE HASH              BRONZE 8094           NaN   NaN   NaN   NaN      NaN   
...                                           ...   ...   ...   ...      ...   
Bismillah K.G.N Express BRONZE 1027117        NaN   NaN   NaN   NaN      NaN   
The_tree nursery        BRONZE 1039630        NaN   NaN   NaN   NaN      NaN   
Ganapati Bags           BRONZE 1047038        NaN   NaN   NaN   NaN      NaN   
Genius Infotech         BRONZE 1049176        NaN   NaN   NaN   NaN      NaN   
Bro Ministry            BRONZE 1052324        NaN   NaN   NaN   NaN      NaN   

                                                                      ...  \
Month                                                       December  ...   
Week                                      week2 week3 week4    week1  ...   
Company Name            Tier   company_id                             ...   
Tanvi Herbals           SILVER 809         74.0  76.0  75.0     74.0  ...   
Mistiq Ark              BRONZE 2311         NaN   NaN   NaN      NaN  ...   
GRANDSELL               BRONZE 3023         NaN   NaN   NaN      NaN  ...   
elizona.com             BRONZE 7389         NaN   NaN   NaN      NaN  ...   
SLATE HASH              BRONZE 8094         NaN   NaN   NaN      NaN  ...   
...                                         ...   ...   ...      ...  ...   
Bismillah K.G.N Express BRONZE 1027117      NaN   NaN   NaN      NaN  ...   
The_tree nursery        BRONZE 1039630      NaN   NaN   NaN      NaN  ...   
Ganapati Bags           BRONZE 1047038      NaN   NaN   NaN      NaN  ...   
Genius Infotech         BRONZE 1049176      NaN   NaN   NaN      NaN  ...   
Bro Ministry            BRONZE 1052324      NaN   NaN   NaN      NaN  ...   

                                          total_attempted_shipments           \
Month                                                       October November   
Week                                                          week4    week1   
Company Name            Tier   company_id                                      
Tanvi Herbals           SILVER 809                            212.0    187.0   
Mistiq Ark              BRONZE 2311                             NaN  

In [21]:
df_ndr= pd.read_excel('bb_report.xlsx',sheet_name='ndr')
df1_ndr=df_ndr

In [22]:
ndr_summary = df1_ndr.groupby(['assigned_month','assigned_week']).agg({'total_shipments':'sum',
                                                                       'total_attempted_shipments':'sum',
                                                                       'delivered_shipment':'sum',
                                                                       'first_attempt_deliver':'sum',
                                                                       'ndr_raised':'sum', 
                                                                        'first_ndr_to_reattempt':'sum',
                                                                        'ndr_raised_delivered':'sum',
                                                                        'ndr_raised_rto':'sum'})
ndr_summary=ndr_summary.reset_index()
ndr_summary = ndr_summary[ndr_summary.assigned_week != "Total"]

ndr_summary['delivered_shipment percent'] = round(ndr_summary['delivered_shipment']*100/ndr_summary['total_attempted_shipments'])
ndr_summary['first_attempt_deliver percent'] = round(ndr_summary['first_attempt_deliver']*100/ndr_summary['total_attempted_shipments'])
ndr_summary['ndr_raised percent'] = round(ndr_summary['ndr_raised']*100/ndr_summary['total_attempted_shipments'])
ndr_summary['first_ndr_to_reattempt percent'] = round(ndr_summary['first_ndr_to_reattempt']*100/ndr_summary['ndr_raised'])
ndr_summary['ndr_raised_delivered percent'] = round(ndr_summary['ndr_raised_delivered']*100/ndr_summary['ndr_raised'])
ndr_summary['ndr_raised_rto percent'] = round(ndr_summary['ndr_raised_rto']*100/ndr_summary['ndr_raised'])

ndr_summary = ndr_summary.reset_index()
ndr_summary = ndr_summary[ndr_summary.assigned_week != "Total"]
ndr_summary = ndr_summary.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
ndr_summary=ndr_summary.sort_values(by='Month', ascending=True)
ndr_summary["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
cols_final=['Month',
 'Week',
 'total_shipments',
 'total_attempted_shipments',
 'delivered_shipment',
 'delivered_shipment percent',
 'first_attempt_deliver',
 'first_attempt_deliver percent',
 'ndr_raised',
  'ndr_raised percent',
 'first_ndr_to_reattempt',
  'first_ndr_to_reattempt percent',
 'ndr_raised_delivered',
  'ndr_raised_delivered percent',
 'ndr_raised_rto',
  'ndr_raised_rto percent']
ndr_summary=ndr_summary[cols_final]
ndr_summary

,Month,Week,total_shipments,total_attempted_shipments,delivered_shipment,delivered_shipment percent,first_attempt_deliver,first_attempt_deliver percent,ndr_raised,ndr_raised percent,first_ndr_to_reattempt,first_ndr_to_reattempt percent,ndr_raised_delivered,ndr_raised_delivered percent,ndr_raised_rto,ndr_raised_rto percent
0,September,week4,6253,6220,4404,71.0,3826,62.0,2171.0,35.0,1538,71.0,578,27.0,1580,73.0
1,October,week1,5388,5356,3946,74.0,3498,65.0,1857.0,35.0,1332,72.0,448,24.0,1371,74.0
2,October,week2,4694,4664,2932,63.0,2477,53.0,2184.0,47.0,1646,75.0,455,21.0,1704,78.0
3,October,week3,4798,4778,2746,57.0,2333,49.0,2442.0,51.0,1727,71.0,413,17.0,2004,82.0
4,October,week4,6105,6075,3017,50.0,2558,42.0,3511.0,58.0,2451,70.0,459,13.0,3028,86.0
5,November,week1,4555,4531,2407,53.0,2092,46.0,2430.0,54.0,1791,74.0,315,13.0,2108,87.0
6,November,week2,3935,3907,2234,57.0,1842,47.0,2063.0,53.0,1544,75.0,392,19.0,1668,81.0
7,November,week3,2734,2723,1652,61.0,1384,51.0,1338.0,49.0,925,69.0,268,20.0,1068,80.0
8,November,week4,2741,2725,1773,65.0,1479,54.0,1242.0,46.0,892,72.0,294,24.0,942,76.0
9,December,week1,1849,1825,962,53.0,791,43.0,1031.0,56.0,772,75.0,171,17.0,850,82.0


In [23]:
df1_ndr_pre = df1_ndr[df1_ndr.payment_mode != "cod"]
ndr_pre_summary = df1_ndr_pre.groupby(['assigned_month','assigned_week']).agg({'total_shipments':'sum',
                                                                       'total_attempted_shipments':'sum',
                                                                       'delivered_shipment':'sum',
                                                                       'first_attempt_deliver':'sum',
                                                                       'ndr_raised':'sum', 
                                                                        'first_ndr_to_reattempt':'sum',
                                                                        'ndr_raised_delivered':'sum',
                                                                        'ndr_raised_rto':'sum'})
ndr_pre_summary=ndr_pre_summary.reset_index()
ndr_pre_summary = ndr_pre_summary[ndr_pre_summary.assigned_week != "Total"]

ndr_pre_summary['delivered_shipment percent'] = round(ndr_pre_summary['delivered_shipment']*100/ndr_pre_summary['total_attempted_shipments'])
ndr_pre_summary['first_attempt_deliver percent'] = round(ndr_pre_summary['first_attempt_deliver']*100/ndr_pre_summary['total_attempted_shipments'])
ndr_pre_summary['ndr_raised percent'] = round(ndr_pre_summary['ndr_raised']*100/ndr_pre_summary['total_attempted_shipments'])
ndr_pre_summary['first_ndr_to_reattempt percent'] = round(ndr_pre_summary['first_ndr_to_reattempt']*100/ndr_pre_summary['ndr_raised'])
ndr_pre_summary['ndr_raised_delivered percent'] = round(ndr_pre_summary['ndr_raised_delivered']*100/ndr_pre_summary['ndr_raised'])
ndr_pre_summary['ndr_raised_rto percent'] = round(ndr_pre_summary['ndr_raised_rto']*100/ndr_pre_summary['ndr_raised'])

ndr_pre_summary = ndr_pre_summary.reset_index()
ndr_pre_summary = ndr_pre_summary[ndr_pre_summary.assigned_week != "Total"]
ndr_pre_summary = ndr_pre_summary.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
ndr_pre_summary=ndr_pre_summary.sort_values(by='Month', ascending=True)
ndr_pre_summary["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
cols_final=['Month',
 'Week',
 'total_shipments',
 'total_attempted_shipments',
 'delivered_shipment',
 'delivered_shipment percent',
 'first_attempt_deliver',
 'first_attempt_deliver percent',
 'ndr_raised',
  'ndr_raised percent',
 'first_ndr_to_reattempt',
  'first_ndr_to_reattempt percent',
 'ndr_raised_delivered',
  'ndr_raised_delivered percent',
 'ndr_raised_rto',
  'ndr_raised_rto percent']
ndr_pre_summary=ndr_pre_summary[cols_final]
ndr_pre_summary

,Month,Week,total_shipments,total_attempted_shipments,delivered_shipment,delivered_shipment percent,first_attempt_deliver,first_attempt_deliver percent,ndr_raised,ndr_raised percent,first_ndr_to_reattempt,first_ndr_to_reattempt percent,ndr_raised_delivered,ndr_raised_delivered percent,ndr_raised_rto,ndr_raised_rto percent
0,September,week4,1593,1592,1572,99.0,1505,95.0,87.0,5.0,81,93.0,67,77.0,19,22.0
1,October,week1,1959,1950,1930,99.0,1834,94.0,116.0,6.0,109,94.0,96,83.0,18,16.0
2,October,week2,679,674,659,98.0,622,92.0,52.0,8.0,46,88.0,37,71.0,14,27.0
3,October,week3,727,727,722,99.0,695,96.0,32.0,4.0,31,97.0,27,84.0,5,16.0
4,October,week4,609,609,603,99.0,571,94.0,38.0,6.0,37,97.0,32,84.0,6,16.0
5,November,week1,683,683,675,99.0,637,93.0,46.0,7.0,43,93.0,38,83.0,8,17.0
6,November,week2,492,488,483,99.0,452,93.0,36.0,7.0,36,100.0,31,86.0,5,14.0
7,November,week3,278,278,275,99.0,262,94.0,16.0,6.0,15,94.0,13,81.0,2,12.0
8,November,week4,367,366,363,99.0,342,93.0,24.0,7.0,24,100.0,21,88.0,3,12.0
9,December,week1,229,229,229,100.0,210,92.0,19.0,8.0,19,100.0,19,100.0,0,0.0


In [24]:
df1_ndr_cod = df1_ndr[df1_ndr.payment_mode != "prepaid"]
ndr_cod_summary= df1_ndr_cod.groupby(['assigned_month','assigned_week']).agg({'total_shipments':'sum',
                                                                       'total_attempted_shipments':'sum',
                                                                       'delivered_shipment':'sum',
                                                                       'first_attempt_deliver':'sum',
                                                                       'ndr_raised':'sum', 
                                                                        'first_ndr_to_reattempt':'sum',
                                                                        'ndr_raised_delivered':'sum',
                                                                        'ndr_raised_rto':'sum'})
ndr_cod_summary=ndr_cod_summary.reset_index()
ndr_cod_summary= ndr_cod_summary[ndr_cod_summary.assigned_week != "Total"]

ndr_cod_summary['delivered_shipment percent'] = round(ndr_cod_summary['delivered_shipment']*100/ndr_cod_summary['total_attempted_shipments'])
ndr_cod_summary['first_attempt_deliver percent'] = round(ndr_cod_summary['first_attempt_deliver']*100/ndr_cod_summary['total_attempted_shipments'])
ndr_cod_summary['ndr_raised percent'] = round(ndr_cod_summary['ndr_raised']*100/ndr_cod_summary['total_attempted_shipments'])
ndr_cod_summary['first_ndr_to_reattempt percent'] = round(ndr_cod_summary['first_ndr_to_reattempt']*100/ndr_cod_summary['ndr_raised'])
ndr_cod_summary['ndr_raised_delivered percent'] = round(ndr_cod_summary['ndr_raised_delivered']*100/ndr_cod_summary['ndr_raised'])
ndr_cod_summary['ndr_raised_rto percent'] = round(ndr_cod_summary['ndr_raised_rto']*100/ndr_cod_summary['ndr_raised'])

ndr_cod_summary= ndr_cod_summary.reset_index()
ndr_cod_summary= ndr_cod_summary[ndr_cod_summary.assigned_week != "Total"]
ndr_cod_summary= ndr_cod_summary.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
ndr_cod_summary=ndr_cod_summary.sort_values(by='Month', ascending=True)
ndr_cod_summary["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
cols_final=['Month',
 'Week',
 'total_shipments',
 'total_attempted_shipments',
 'delivered_shipment',
 'delivered_shipment percent',
 'first_attempt_deliver',
 'first_attempt_deliver percent',
 'ndr_raised',
  'ndr_raised percent',
 'first_ndr_to_reattempt',
  'first_ndr_to_reattempt percent',
 'ndr_raised_delivered',
  'ndr_raised_delivered percent',
 'ndr_raised_rto',
  'ndr_raised_rto percent']
ndr_cod_summary=ndr_cod_summary[cols_final]
ndr_cod_summary

,Month,Week,total_shipments,total_attempted_shipments,delivered_shipment,delivered_shipment percent,first_attempt_deliver,first_attempt_deliver percent,ndr_raised,ndr_raised percent,first_ndr_to_reattempt,first_ndr_to_reattempt percent,ndr_raised_delivered,ndr_raised_delivered percent,ndr_raised_rto,ndr_raised_rto percent
0,September,week4,4660,4628,2832,61.0,2321,50.0,2084.0,45.0,1457,70.0,511,25.0,1561,75.0
1,October,week1,3429,3406,2016,59.0,1664,49.0,1741.0,51.0,1223,70.0,352,20.0,1353,78.0
2,October,week2,4015,3990,2273,57.0,1855,46.0,2132.0,53.0,1600,75.0,418,20.0,1690,79.0
3,October,week3,4071,4051,2024,50.0,1638,40.0,2410.0,59.0,1696,70.0,386,16.0,1999,83.0
4,October,week4,5496,5466,2414,44.0,1987,36.0,3473.0,64.0,2414,70.0,427,12.0,3022,87.0
5,November,week1,3872,3848,1732,45.0,1455,38.0,2384.0,62.0,1748,73.0,277,12.0,2100,88.0
6,November,week2,3443,3419,1751,51.0,1390,41.0,2027.0,59.0,1508,74.0,361,18.0,1663,82.0
7,November,week3,2456,2445,1377,56.0,1122,46.0,1322.0,54.0,910,69.0,255,19.0,1066,81.0
8,November,week4,2374,2359,1410,60.0,1137,48.0,1218.0,52.0,868,71.0,273,22.0,939,77.0
9,December,week1,1620,1596,733,46.0,581,36.0,1012.0,63.0,753,74.0,152,15.0,850,84.0


In [25]:
ndr_cid = df1_ndr.groupby(['assigned_month','assigned_week','company_id']).agg({'total_shipments':'sum',
                                                                       'total_attempted_shipments':'sum',
                                                                       'delivered_shipment':'sum',
                                                                       'first_attempt_deliver':'sum',
                                                                       'ndr_raised':'sum', 
                                                                        'first_ndr_to_reattempt':'sum',
                                                                        'ndr_raised_delivered':'sum',
                                                                        'ndr_raised_rto':'sum'})
ndr_cid=ndr_cid.reset_index()
ndr_cid = ndr_cid[ndr_cid.assigned_week != "Total"]

ndr_cid['delivered_shipment percent'] = round(ndr_cid['delivered_shipment']*100/ndr_cid['total_attempted_shipments'])
ndr_cid['first_attempt_deliver percent'] = round(ndr_cid['first_attempt_deliver']*100/ndr_cid['total_attempted_shipments'])
ndr_cid['ndr_raised percent'] = round(ndr_cid['ndr_raised']*100/ndr_cid['total_attempted_shipments'])
ndr_cid['first_ndr_to_reattempt percent'] = round(ndr_cid['first_ndr_to_reattempt']*100/ndr_cid['ndr_raised'])
ndr_cid['ndr_raised_delivered percent'] = round(ndr_cid['ndr_raised_delivered']*100/ndr_cid['ndr_raised'])
ndr_cid['ndr_raised_rto percent'] = round(ndr_cid['ndr_raised_rto']*100/ndr_cid['ndr_raised'])

ndr_cid = ndr_cid.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
ndr_cid=ndr_cid.sort_values(by='Month', ascending=True)
ndr_cid["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
ndr_cid_final = ndr_cid.pivot_table([ 'delivered_shipment percent','total_attempted_shipments','first_attempt_deliver percent',
 'ndr_raised percent',
 'first_ndr_to_reattempt percent',
 'ndr_raised_rto percent'], ['company_id'],['Month', 'Week'])
ndr_cid_final = ndr_cid_final.reset_index()
ndr_cid_final = ndr_cid_final.sort_values('company_id')
ndr_cid_final['company_id'] = ndr_cid_final['company_id'].astype(int)
ndr_cid_final = ndr_cid_final.sort_values('company_id')
# ndr_cid_final['Company Name']=cnames
# ndr_cid_final['Tier']=ctier
ndr_cid_final.insert(0, 'Tier', ctier)
ndr_cid_final.insert(0, 'Company Name', cnames)

In [26]:
ndr_cid_final = ndr_cid_final.reset_index()
# ndr_cid_final = ndr_cid_final.set_index('Company Name')
ndr_cid_final = ndr_cid_final.set_index(['Company Name', 'Tier','company_id'])
# ndr_cid_final.drop(['index'], axis='columns', inplace=True)
ndr_cid_final = ndr_cid_final.reindex(cols_order, axis=1, level=1)
ndr_cid_final

delivered_shipment percent          \
Month                                                      September October   
Week                                                           week4   week1   
Company Name            Tier   company_id                                      
Tanvi Herbals           SILVER 809                              86.0    84.0   
Mistiq Ark              BRONZE 2311                              NaN     NaN   
GRANDSELL               BRONZE 3023                              NaN     NaN   
elizona.com             BRONZE 7389                              NaN     NaN   
SLATE HASH              BRONZE 8094                              NaN     NaN   
...                                                              ...     ...   
Bismillah K.G.N Express BRONZE 1027117                           NaN     NaN   
The_tree nursery        BRONZE 1039630                           NaN     NaN   
Ganapati Bags           BRONZE 1047038                           NaN     NaN   
Genius Infotech         BRONZE 1049176                           NaN     NaN   
Bro Ministry            BRONZE 1052324                           NaN     NaN   

                                                                            \
Month                                                       November         
Week                                      week2 week3 week4    week1 week2   
Company Name            Tier   company_id                                    
Tanvi Herbals           SILVER 809         82.0  86.0  84.0     84.0  85.0   
Mistiq Ark              BRONZE 2311         NaN   NaN   NaN      NaN   NaN   
GRANDSELL               BRONZE 3023         NaN   NaN   NaN      NaN   NaN   
elizona.com             BRONZE 7389         NaN   NaN   NaN      NaN   NaN   
SLATE HASH              BRONZE 8094         NaN   NaN   NaN      NaN   NaN   
...                                         ...   ...   ...      ...   ...   
Bismillah K.G.N Express BRONZE 1027117      NaN   NaN   NaN      NaN   NaN   
The_tree nursery        BRONZE 1039630      NaN   NaN   NaN      NaN   NaN   
Ganapati Bags           BRONZE 1047038      NaN   NaN   NaN      NaN   NaN   
Genius Infotech         BRONZE 1049176      NaN   NaN   NaN      NaN   NaN   
Bro Ministry            BRONZE 1052324      NaN   NaN   NaN      NaN   NaN   

                                                                ...  \
Month                                                 December  ...   
Week                                      week3 week4    week1  ...   
Company Name            Tier   company_id                       ...   
Tanvi Herbals           SILVER 809         86.0  87.0     85.0  ...   
Mistiq Ark              BRONZE 2311         NaN   NaN      NaN  ...   
GRANDSELL               BRONZE 3023         NaN   NaN      NaN  ...   
elizona.com             BRONZE 7389         NaN   NaN      NaN  ...   
SLATE HASH              BRONZE 8094         NaN   NaN      NaN  ...   
...                                         ...   ...      ...  ...   
Bismillah K.G.N Express BRONZE 1027117      NaN   NaN      NaN  ...   
The_tree nursery        BRONZE 1039630      NaN   NaN      NaN  ...   
Ganapati Bags           BRONZE 1047038      NaN   NaN      NaN  ...   
Genius Infotech         BRONZE 1049176      NaN   NaN      NaN  ...   
Bro Ministry            BRONZE 1052324      NaN   NaN      NaN  ...   

                                          total_attempted_shipments           \
Month                                                       October November   
Week                                                          week4    week1   
Company Name            Tier   company_id                                      
Tanvi Herbals           SILVER 809                            212.0    187.0   
Mistiq Ark              BRONZE 2311                             NaN      NaN   
GRANDSELL               BRONZE 3023                             NaN      NaN   
elizona.com             BRONZE 7389         

In [27]:
ndr_cid_pre = df1_ndr_pre.groupby(['assigned_month','assigned_week','company_id']).agg({'total_shipments':'sum',
                                                                       'total_attempted_shipments':'sum',
                                                                       'delivered_shipment':'sum',
                                                                       'first_attempt_deliver':'sum',
                                                                       'ndr_raised':'sum', 
                                                                        'first_ndr_to_reattempt':'sum',
                                                                        'ndr_raised_delivered':'sum',
                                                                        'ndr_raised_rto':'sum'})
ndr_cid_pre=ndr_cid_pre.reset_index()
ndr_cid_pre = ndr_cid_pre[ndr_cid_pre.assigned_week != "Total"]

ndr_cid_pre['delivered_shipment percent'] = round(ndr_cid_pre['delivered_shipment']*100/ndr_cid_pre['total_attempted_shipments'])
ndr_cid_pre['first_attempt_deliver percent'] = round(ndr_cid_pre['first_attempt_deliver']*100/ndr_cid_pre['total_attempted_shipments'])
ndr_cid_pre['ndr_raised percent'] = round(ndr_cid_pre['ndr_raised']*100/ndr_cid_pre['total_attempted_shipments'])
ndr_cid_pre['first_ndr_to_reattempt percent'] = round(ndr_cid_pre['first_ndr_to_reattempt']*100/ndr_cid_pre['ndr_raised'])
ndr_cid_pre['ndr_raised_delivered percent'] = round(ndr_cid_pre['ndr_raised_delivered']*100/ndr_cid_pre['ndr_raised'])
ndr_cid_pre['ndr_raised_rto percent'] = round(ndr_cid_pre['ndr_raised_rto']*100/ndr_cid_pre['ndr_raised'])

ndr_cid_pre = ndr_cid_pre.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
ndr_cid_pre=ndr_cid_pre.sort_values(by='Month', ascending=True)
ndr_cid_pre["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
ndr_cid_pre_final = ndr_cid_pre.pivot_table([ 'delivered_shipment percent','first_attempt_deliver percent','total_attempted_shipments',
 'ndr_raised percent',
 'first_ndr_to_reattempt percent',
 'ndr_raised_rto percent'], ['company_id'],['Month', 'Week'])
ndr_cid_pre_final = ndr_cid_pre_final.reset_index()
ndr_cid_pre_final = ndr_cid_pre_final.sort_values('company_id')
ndr_cid_pre_final['company_id'] = ndr_cid_pre_final['company_id'].astype(int)
ndr_cid_pre_final = ndr_cid_pre_final.sort_values('company_id')

In [28]:
ndr_cid_pre_final = ndr_cid_pre_final.reset_index()
ndr_cid_pre_final = ndr_cid_pre_final.set_index('company_id')
ndr_cid_pre_final.drop(['index'], axis='columns', inplace=True)
ndr_cid_pre_final = ndr_cid_pre_final.reindex(cols_order, axis=1, level=1)
ndr_cid_pre_final

C:\Users\abc\anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


delivered_shipment percent                                      \
Month                       September October                    November   
Week                            week4   week1 week2  week3 week4    week1   
company_id                                                                  
809                             100.0    99.0  99.0  100.0  99.0    100.0   
2311                              NaN     NaN   NaN    NaN   NaN      NaN   
3023                              NaN     NaN   NaN    NaN   NaN      NaN   
7389                              NaN     NaN   NaN    NaN   NaN      NaN   
8094                              NaN     NaN   NaN    NaN   NaN      NaN   
...                               ...     ...   ...    ...   ...      ...   
1022128                           NaN     NaN   NaN    NaN   NaN      NaN   
1027117                           NaN     NaN   NaN    NaN   NaN      NaN   
1039630                           NaN     NaN   NaN    NaN   NaN      NaN   
1047038                           NaN     NaN   NaN    NaN   NaN      NaN   
1049176                           NaN     NaN   NaN    NaN   NaN      NaN   

                                        ... total_attempted_shipments  \
Month                         December  ...                   October   
Week       week2  week3 week4    week1  ...                     week4   
company_id                              ...                             
809         99.0  100.0  99.0    100.0  ...                     118.0   
2311         NaN    NaN   NaN      NaN  ...                       NaN   
3023         NaN    NaN   NaN      NaN  ...                       NaN   
7389         NaN    NaN   NaN      NaN  ...                       NaN   
8094         NaN    NaN   NaN      NaN  ...                       NaN   
...          ...    ...   ...      ...  ...                       ...   
1022128      NaN    NaN   NaN      NaN  ...                       NaN   
1027117      NaN    NaN   NaN      NaN  ...                       NaN   
1039630      NaN    NaN   NaN      NaN  ...                       NaN   
1047038      NaN    NaN   NaN      NaN  ...                       NaN   
1049176      NaN    NaN   NaN      NaN  ...                       NaN   

                                                                           
Month      November                    December                   January  
Week          week1 week2 week3  week4    week1 week2 week3 week4   week1  
company_id                                                                 
809            89.0  72.0  73.0  122.0     94.0  90.0  92.0  42.0     NaN  
2311            NaN   NaN   NaN    NaN      NaN   NaN   NaN   1.0     NaN  
3023            NaN   NaN   NaN    NaN      NaN   NaN   NaN   1.0     0.0  
7389            NaN   NaN   NaN    NaN      NaN   NaN   NaN   3.0     0.0  
8094            NaN   NaN   NaN    NaN      NaN   NaN   NaN   0.0     NaN  
...             ...   ...   ...    ...      ...   ...   ...   ...     ...  
1022128         NaN   NaN   NaN    NaN      NaN   NaN   NaN   2.0     0.0  
1027117         NaN   NaN   NaN    NaN      NaN   NaN   NaN  13.0     NaN  
1039630         NaN   NaN   NaN    NaN      NaN   NaN   NaN   4.0     1.0  
1047038         NaN   NaN   NaN    NaN      NaN   NaN   NaN   2.0     NaN  
1049176         NaN   NaN   NaN    NaN      NaN   NaN   NaN   1.0     0.0  

[216 rows x 84 columns]

In [29]:
ndr_cid_cod = df1_ndr_cod.groupby(['assigned_month','assigned_week','company_id']).agg({'total_shipments':'sum',
                                                                       'total_attempted_shipments':'sum',
                                                                       'delivered_shipment':'sum',
                                                                       'first_attempt_deliver':'sum',
                                                                       'ndr_raised':'sum', 
                                                                        'first_ndr_to_reattempt':'sum',
                                                                        'ndr_raised_delivered':'sum',
                                                                        'ndr_raised_rto':'sum'})
ndr_cid_cod=ndr_cid_cod.reset_index()
ndr_cid_cod = ndr_cid_cod[ndr_cid_cod.assigned_week != "Total"]

ndr_cid_cod['delivered_shipment percent'] = round(ndr_cid_cod['delivered_shipment']*100/ndr_cid_cod['total_attempted_shipments'])
ndr_cid_cod['first_attempt_deliver percent'] = round(ndr_cid_cod['first_attempt_deliver']*100/ndr_cid_cod['total_attempted_shipments'])
ndr_cid_cod['ndr_raised percent'] = round(ndr_cid_cod['ndr_raised']*100/ndr_cid_cod['total_attempted_shipments'])
ndr_cid_cod['first_ndr_to_reattempt percent'] = round(ndr_cid_cod['first_ndr_to_reattempt']*100/ndr_cid_cod['ndr_raised'])
ndr_cid_cod['ndr_raised_delivered percent'] = round(ndr_cid_cod['ndr_raised_delivered']*100/ndr_cid_cod['ndr_raised'])
ndr_cid_cod['ndr_raised_rto percent'] = round(ndr_cid_cod['ndr_raised_rto']*100/ndr_cid_cod['ndr_raised'])

ndr_cid_cod = ndr_cid_cod.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
# ndr_cid_cod["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December"}, inplace=True)
ndr_cid_cod=ndr_cid_cod.sort_values(by='Month', ascending=True)
ndr_cid_cod["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
ndr_cid_cod_final = ndr_cid_cod.pivot_table([ 'delivered_shipment percent','first_attempt_deliver percent','total_attempted_shipments',
 'ndr_raised percent',
 'first_ndr_to_reattempt percent',
 'ndr_raised_rto percent'], ['company_id'],['Month', 'Week'])
ndr_cid_cod_final = ndr_cid_cod_final.reset_index()
ndr_cid_cod_final = ndr_cid_cod_final.sort_values('company_id')
ndr_cid_cod_final['company_id'] = ndr_cid_cod_final['company_id'].astype(int)
ndr_cid_cod_final = ndr_cid_cod_final.sort_values('company_id')
ndr_cid_cod_final

company_id delivered_shipment percent                             \
Month                              December                    January   
Week                                  week1 week2 week3  week4   week1   
128          809                       68.0  76.0  76.0   84.0     NaN   
113         7389                        NaN   NaN   NaN  100.0     NaN   
142         8880                        NaN   NaN   NaN   86.0     NaN   
8          11602                        NaN   NaN   NaN   52.0     0.0   
13         15382                        NaN   NaN   NaN  100.0     NaN   
..           ...                        ...   ...   ...    ...     ...   
1        1010615                        NaN   NaN   NaN   50.0     NaN   
2        1016834                        NaN   NaN   NaN   54.0    55.0   
3        1020207                        NaN   NaN   NaN  100.0     NaN   
4        1022128                        NaN   NaN   NaN  100.0     NaN   
5        1052324                        NaN   NaN   NaN   44.0     NaN   

                                  ... total_attempted_shipments           \
Month November                    ...                   January November   
Week     week1 week2 week3 week4  ...                     week1    week1   
128       70.0  74.0  73.0  74.0  ...                       NaN     98.0   
113        NaN   NaN   NaN   NaN  ...                       NaN      NaN   
142        NaN   NaN   NaN   NaN  ...                       0.0      NaN   
8          NaN   NaN   NaN   NaN  ...                       1.0      NaN   
13         NaN   NaN   NaN   NaN  ...                       NaN      NaN   
..         ...   ...   ...   ...  ...                       ...      ...   
1          NaN   NaN   NaN   NaN  ...                       NaN      NaN   
2          NaN   NaN   NaN   NaN  ...                      22.0      NaN   
3          NaN   NaN   NaN   NaN  ...                       NaN      NaN   
4          NaN   NaN   NaN   NaN  ...                       0.0      NaN   
5          NaN   NaN   NaN   NaN  ...                       NaN      NaN   

                                                               
Month                    October                    September  
Week  week2 week3  week4   week1  week2 week3 week4     week4  
128    87.0  75.0  117.0    90.0  116.0  73.0  94.0     101.0  
113     NaN   NaN    NaN     NaN    NaN   NaN   NaN       NaN  
142     NaN   NaN    NaN     NaN    NaN   NaN   NaN       NaN  
8       NaN   NaN    NaN     NaN    NaN   NaN   NaN       NaN  
13      NaN   NaN    NaN     NaN    NaN   NaN   NaN       NaN  
..      ...   ...    ...     ...    ...   ...   ...       ...  
1       NaN   NaN    NaN     NaN    NaN   NaN   NaN       NaN  
2       NaN   NaN    NaN     NaN    NaN   NaN   NaN       NaN  
3       NaN   NaN    NaN     NaN    NaN   NaN   NaN       NaN  
4       NaN   NaN    NaN     NaN    NaN   NaN   NaN       NaN  
5       NaN   NaN    NaN     NaN    NaN   NaN   NaN       NaN  

[153 rows x 85 columns]

In [30]:
ndr_cid_cod_final = ndr_cid_cod_final.reset_index()
ndr_cid_cod_final = ndr_cid_cod_final.set_index('company_id')
ndr_cid_cod_final.drop(['index'], axis='columns', inplace=True)
ndr_cid_cod_final = ndr_cid_cod_final.reindex(cols_order, axis=1, level=1)
ndr_cid_cod_final

C:\Users\abc\anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


delivered_shipment percent                                     \
Month                       September October                   November   
Week                            week4   week1 week2 week3 week4    week1   
company_id                                                                 
809                              75.0    68.0  72.0  73.0  66.0     70.0   
7389                              NaN     NaN   NaN   NaN   NaN      NaN   
8880                              NaN     NaN   NaN   NaN   NaN      NaN   
11602                             NaN     NaN   NaN   NaN   NaN      NaN   
15382                             NaN     NaN   NaN   NaN   NaN      NaN   
...                               ...     ...   ...   ...   ...      ...   
1010615                           NaN     NaN   NaN   NaN   NaN      NaN   
1016834                           NaN     NaN   NaN   NaN   NaN      NaN   
1020207                           NaN     NaN   NaN   NaN   NaN      NaN   
1022128                           NaN     NaN   NaN   NaN   NaN      NaN   
1052324                           NaN     NaN   NaN   NaN   NaN      NaN   

                                       ... total_attempted_shipments           \
Month                        December  ...                   October November   
Week       week2 week3 week4    week1  ...                     week4    week1   
company_id                             ...                                      
809         74.0  73.0  74.0     68.0  ...                      94.0     98.0   
7389         NaN   NaN   NaN      NaN  ...                       NaN      NaN   
8880         NaN   NaN   NaN      NaN  ...                       NaN      NaN   
11602        NaN   NaN   NaN      NaN  ...                       NaN      NaN   
15382        NaN   NaN   NaN      NaN  ...                       NaN      NaN   
...          ...   ...   ...      ...  ...                       ...      ...   
1010615      NaN   NaN   NaN      NaN  ...                       NaN      NaN   
1016834      NaN   NaN   NaN      NaN  ...                       NaN      NaN   
1020207      NaN   NaN   NaN      NaN  ...                       NaN      NaN   
1022128      NaN   NaN   NaN      NaN  ...                       NaN      NaN   
1052324      NaN   NaN   NaN      NaN  ...                       NaN      NaN   

                                                                   
Month                         December                    January  
Week       week2 week3  week4    week1 week2 week3  week4   week1  
company_id                                                         
809         87.0  75.0  117.0     88.0  85.0  72.0   44.0     NaN  
7389         NaN   NaN    NaN      NaN   NaN   NaN    1.0     NaN  
8880         NaN   NaN    NaN      NaN   NaN   NaN    7.0     0.0  
11602        NaN   NaN    NaN      NaN   NaN   NaN   67.0     1.0  
15382        NaN   NaN    NaN      NaN   NaN   NaN    4.0     NaN  
...          ...   ...    ...      ...   ...   ...    ...     ...  
1010615      NaN   NaN    NaN      NaN   NaN   NaN   36.0     NaN  
1016834      NaN   NaN    NaN      NaN   NaN   NaN  322.0    22.0  
1020207      NaN   NaN    NaN      NaN   NaN   NaN    1.0     NaN  
1022128      NaN   NaN    NaN      NaN   NaN   NaN    2.0     0.0  
1052324      NaN   NaN    NaN      NaN   NaN   NaN   27.0     NaN  

[153 rows x 84 columns]

In [31]:
df_previous_ndr= pd.read_excel('prev_bb_companies.xlsx',sheet_name='ndr')
df1_previous_ndr=df_previous_ndr

In [32]:
#delivery performance compare
#COD
df1_ndr_cod_comp = df1_ndr[df1_ndr.payment_mode != "prepaid"]
ndr_cid_cod_comp = df1_ndr_cod_comp.groupby(['assigned_month','company_id']).agg({'total_attempted_shipments':'sum',
                                                                       'delivered_shipment':'sum'})
ndr_cid_cod_comp=ndr_cid_cod_comp.reset_index()
ndr_cid_cod_comp = ndr_cid_cod_comp[ndr_cid_cod_comp.company_id != "Total"]
ndr_cid_cod_comp['delivered_shipment percent'] = round(ndr_cid_cod_comp['delivered_shipment']*100/ndr_cid_cod_comp['total_attempted_shipments'])
ndr_cid_cod_comp = ndr_cid_cod_comp.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
# ndr_cid_cod_comp["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December"}, inplace=True)
ndr_cid_cod_comp=ndr_cid_cod_comp.sort_values(by='Month', ascending=True)
ndr_cid_cod_comp["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
ndr_cid_cod_comp_final = ndr_cid_cod_comp.pivot_table([ 'delivered_shipment percent'], ['company_id'],['Month'])
ndr_cid_cod_comp_final = ndr_cid_cod_comp_final.reset_index()
ndr_cid_cod_comp_final = ndr_cid_cod_comp_final.sort_values('company_id')
ndr_cid_cod_comp_final['company_id'] = ndr_cid_cod_comp_final['company_id'].astype(int)
ndr_cid_cod_comp_final = ndr_cid_cod_comp_final.sort_values('company_id')
# ndr_cid_cod_comp_final.drop([('delivered_shipment percent', 'September')], axis='columns', inplace=True)
ndr_cid_cod_comp_final = ndr_cid_cod_comp_final.set_index('company_id')

#arrange
# cols_final=[('delivered_shipment percent', 'September'),
#  ('delivered_shipment percent', 'October'),
#             ('delivered_shipment percent', 'November'),
#            ('delivered_shipment percent', 'December')]
# ndr_cid_cod_comp_final=ndr_cid_cod_comp_final[cols_final]

#Prepaid
df1_ndr_pre_comp = df1_ndr[df1_ndr.payment_mode != "cod"]
ndr_cid_pre_comp = df1_ndr_pre_comp.groupby(['assigned_month','company_id']).agg({'total_attempted_shipments':'sum',
                                                                       'delivered_shipment':'sum'})
ndr_cid_pre_comp=ndr_cid_pre_comp.reset_index()
ndr_cid_pre_comp = ndr_cid_pre_comp[ndr_cid_pre_comp.company_id != "Total"]
ndr_cid_pre_comp['delivered_shipment percent'] = round(ndr_cid_pre_comp['delivered_shipment']*100/ndr_cid_pre_comp['total_attempted_shipments'])
ndr_cid_pre_comp = ndr_cid_pre_comp.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
# ndr_cid_pre_comp["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December"}, inplace=True)
ndr_cid_pre_comp=ndr_cid_pre_comp.sort_values(by='Month', ascending=True)
ndr_cid_pre_comp["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
ndr_cid_pre_comp_final = ndr_cid_pre_comp.pivot_table([ 'delivered_shipment percent'], ['company_id'],['Month'])
ndr_cid_pre_comp_final = ndr_cid_pre_comp_final.reset_index()
ndr_cid_pre_comp_final = ndr_cid_pre_comp_final.sort_values('company_id')
ndr_cid_pre_comp_final['company_id'] = ndr_cid_pre_comp_final['company_id'].astype(int)
ndr_cid_pre_comp_final = ndr_cid_pre_comp_final.sort_values('company_id')
ndr_cid_pre_comp_final = ndr_cid_pre_comp_final.set_index('company_id')
# ndr_cid_pre_comp_final=ndr_cid_pre_comp_final[cols_final]

#Previous COD
df1_ndr_cod_prev = df1_previous_ndr[df1_previous_ndr.payment_mode != "prepaid"]
ndr_cid_cod_prev = df1_ndr_cod_prev.groupby(['assigned_month','company_id']).agg({'total_attempted_shipments':'sum',
                                                                       'delivered_shipment':'sum'})
ndr_cid_cod_prev=ndr_cid_cod_prev.reset_index()
ndr_cid_cod_prev = ndr_cid_cod_prev[ndr_cid_cod_prev.company_id != "Total"]
ndr_cid_cod_prev['delivered_shipment percent'] = round(ndr_cid_cod_prev['delivered_shipment']*100/ndr_cid_cod_prev['total_attempted_shipments'])
ndr_cid_cod_prev = ndr_cid_cod_prev.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
# ndr_cid_cod_prev["Month"].replace({202008: "August",202009: "September", 202010: "October", 202011: "November", 202012: "December"}, inplace=True)
ndr_cid_cod_prev=ndr_cid_cod_prev.sort_values(by='Month', ascending=True)
ndr_cid_cod_prev["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
ndr_cid_cod_prev_final = ndr_cid_cod_prev.pivot_table([ 'delivered_shipment percent'], ['company_id'],['Month'])
ndr_cid_cod_prev_final = ndr_cid_cod_prev_final.reset_index()
ndr_cid_cod_prev_final = ndr_cid_cod_prev_final.sort_values('company_id')
ndr_cid_cod_prev_final['company_id'] = ndr_cid_cod_prev_final['company_id'].astype(int)
ndr_cid_cod_prev_final = ndr_cid_cod_prev_final.sort_values('company_id')
ndr_cid_cod_prev_final = ndr_cid_cod_prev_final.set_index('company_id')

#Previous prepaid
df1_ndr_pre_prev = df1_previous_ndr[df1_previous_ndr.payment_mode != "cod"]
ndr_cid_pre_prev = df1_ndr_pre_prev.groupby(['assigned_month','company_id']).agg({'total_attempted_shipments':'sum',
                                                                       'delivered_shipment':'sum'})
ndr_cid_pre_prev=ndr_cid_pre_prev.reset_index()
ndr_cid_pre_prev = ndr_cid_pre_prev[ndr_cid_pre_prev.company_id != "Total"]
ndr_cid_pre_prev['delivered_shipment percent'] = round(ndr_cid_pre_prev['delivered_shipment']*100/ndr_cid_pre_prev['total_attempted_shipments'])
ndr_cid_pre_prev = ndr_cid_pre_prev.rename(columns={"assigned_month": "Month", "assigned_week": "Week"})
# ndr_cid_pre_prev["Month"].replace({202008: "August",202009: "September", 202010: "October", 202011: "November", 202012: "December"}, inplace=True)
ndr_cid_pre_prev=ndr_cid_pre_prev.sort_values(by='Month', ascending=True)
ndr_cid_pre_prev["Month"].replace({202009: "September", 202010: "October", 202011: "November", 202012: "December",202101: "January",
                                 202102: "February",202103: "March",202104: "April",202105: "May",202106: "June",
                                 202107: "July",202108: "August"}, inplace=True)
ndr_cid_pre_prev_final = ndr_cid_pre_prev.pivot_table([ 'delivered_shipment percent'], ['company_id'],['Month'])
ndr_cid_pre_prev_final = ndr_cid_pre_prev_final.reset_index()
ndr_cid_pre_prev_final = ndr_cid_pre_prev_final.sort_values('company_id')
ndr_cid_pre_prev_final['company_id'] = ndr_cid_pre_prev_final['company_id'].astype(int)
ndr_cid_pre_prev_final = ndr_cid_pre_prev_final.sort_values('company_id')
ndr_cid_pre_prev_final = ndr_cid_pre_prev_final.set_index('company_id')

In [33]:
# ndr_cid_cod_comp_final.drop(['delivered_shipment percent', 'September'],axis=1,inplace=True)
# ndr_cid_cod_comp_final.columns=['September','October','November','December']
ndr_cid_cod_prev_final.columns=['August_presmart','September_Presmart']
# ndr_cid_cod_comp_final.drop('September',axis=1,inplace=True)
deliv_comp_cod=ndr_cid_cod_prev_final.merge(ndr_cid_cod_comp_final,on='company_id',how='outer')
# deliv_comp_cod = deliv_comp_cod.reindex(cols_order, axis=0, level=1)
deliv_comp_cod

C:\Users\abc\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,August_presmart,September_Presmart,"(delivered_shipment percent, December)","(delivered_shipment percent, January)","(delivered_shipment percent, November)","(delivered_shipment percent, October)","(delivered_shipment percent, September)"
company_id,,,,,,,
809,62.0,74.0,75.0,NaN,73.0,69.0,75.0
16888,81.0,68.0,NaN,NaN,NaN,68.0,74.0
25149,50.0,100.0,NaN,NaN,NaN,NaN,NaN
28701,48.0,55.0,20.0,NaN,NaN,NaN,NaN
35994,94.0,100.0,88.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1010615,NaN,NaN,50.0,NaN,NaN,NaN,NaN
1016834,NaN,NaN,54.0,55.0,NaN,NaN,NaN
1020207,NaN,NaN,100.0,NaN,NaN,NaN,NaN


In [34]:
# ndr_cid_cod_comp_final.drop(['delivered_shipment percent', 'September'],axis=1,inplace=True)
# ndr_cid_pre_comp_final.columns=['September','October','November','December']
ndr_cid_pre_prev_final.columns=['August_presmart','September_presmart']
# ndr_cid_pre_comp_final.drop('September',axis=1,inplace=True)
deliv_comp_pre=ndr_cid_pre_prev_final.merge(ndr_cid_pre_comp_final,on='company_id',how='outer')
# deliv_comp_pre = deliv_comp_pre.reindex(cols_order, axis=1, level=1)
deliv_comp_pre

C:\Users\abc\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,August_presmart,September_presmart,"(delivered_shipment percent, December)","(delivered_shipment percent, January)","(delivered_shipment percent, November)","(delivered_shipment percent, October)","(delivered_shipment percent, September)"
company_id,,,,,,,
809,99.0,99.0,100.0,NaN,99.0,99.0,100.0
16888,100.0,100.0,NaN,NaN,NaN,100.0,100.0
25149,400.0,125.0,NaN,NaN,NaN,NaN,NaN
28701,98.0,100.0,NaN,NaN,NaN,NaN,NaN
35994,100.0,100.0,100.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1022128,NaN,NaN,100.0,NaN,NaN,NaN,NaN
1027117,NaN,NaN,100.0,NaN,NaN,NaN,NaN
1039630,NaN,NaN,100.0,100.0,NaN,NaN,NaN


In [35]:
#mail summary
# ranking_summary_mail=ranking_summary[(ranking_summary['Month']=='December') | (ranking_summary['Month']=='November')]
ranking_summary_mail=ranking_summary.copy()
ranking_summary_mail.set_index('Month',inplace=True)
ranking_summary_mail=ranking_summary_mail.T

pickup_summary_mail=pickup_summary[['Month','Week','Not Picked (Percentage)','Picked in < 2 days (Percentage)']]
# pickup_summary_mail=pickup_summary_mail[(pickup_summary_mail['Month']=='December')]
pickup_summary_mail.set_index('Month',inplace=True)
pickup_summary_mail=pickup_summary_mail.T


rto_summary_mail=rto_summary[['Month','Week','RTO percentage']]
# rto_summary_mail=rto_summary_mail[(rto_summary_mail['Month']=='December')]
rto_summary_mail.set_index('Month',inplace=True)
rto_summary_mail=rto_summary_mail.T

delivery_summary_mail=delivery_summary[['Month','Week','First_attempt_delivery (Percentage)','Percentage within EDD','avg_first_attempt_time','avg_delivered_time']]
# delivery_summary_mail=delivery_summary_mail[(delivery_summary_mail['Month']=='December')]
delivery_summary_mail.set_index('Month',inplace=True)
delivery_summary_mail=delivery_summary_mail.T

ndr_summary_mail=ndr_summary[['Month','Week','delivered_shipment percent','first_ndr_to_reattempt percent','ndr_raised percent','ndr_raised_delivered percent','ndr_raised_rto percent']]
# ndr_summary_mail=ndr_summary_mail[(ndr_summary_mail['Month']=='December')]
ndr_summary_mail.set_index('Month',inplace=True)
ndr_summary_mail=ndr_summary_mail.T

delivery_summary_mail=delivery_summary_mail[delivery_summary_mail.index!='Week']
ndr_summary_mail=ndr_summary_mail[ndr_summary_mail.index!='Week']
rto_summary_mail=rto_summary_mail[rto_summary_mail.index!='Week']
ranking_summary_mail=ranking_summary_mail[ranking_summary_mail.index!='Week']
df_all = pd.concat([pickup_summary_mail,delivery_summary_mail,ndr_summary_mail,rto_summary_mail,ranking_summary_mail])

In [36]:
# df2 = df.rename(index={0: '#0', 1: '#1', 2: '#2'})
df_all_final = df_all.rename(index={
'First_attempt_delivery (Percentage)':'First Attempt Delivered Percentage',
'Percentage within EDD':'Percentage Within EDD',
 'avg_first_attempt_time':'Average First Attempt Time(In hours)',
'avg_delivered_time':'Average Delivered Time(In hours)',
'delivered_shipment percent':'Percentage of delivered_shipment',
'first_ndr_to_reattempt percent':'Percentage of first_ndr_to_reattempt',
'ndr_raised percent':'Percentage of ndr_raised',
'ndr_raised_delivered percent':'Percentage of ndr_raised_delivered',
'ndr_raised_rto percent':'Percentage of ndr_raised_rto',
'RTO percentage':'RTO Percentage',
'Ranking percentage':'Percentage of Shipments assigned to the first 3 ranks'})
df_all_final

Month,September,October,October,October,October,November,November,November,November,December,December,December,December,January
Week,week4,week1,week2,week3,week4,week1,week2,week3,week4,week1,week2,week3,week4,week1
Not Picked (Percentage),0,0,0,0,0,0,0,0,0,0,0,0,1,20
Picked in < 2 days (Percentage),98,95,97,89,92,96,94,96,97,94,99,98,95,80
First Attempt Delivered Percentage,62,65,53,49,42,46,47,51,54,43,32,34,58,72
Percentage Within EDD,79,89,83,46,76,75,58,76,70,67,70,67,69,93
Average First Attempt Time(In hours),90,91,97,91,94,88,107,86,88,86,91,88,76,40
Average Delivered Time(In hours),95,98,108,101,102,96,115,90,92,93,100,95,81,43
Percentage of delivered_shipment,71,74,63,57,50,53,57,61,65,53,38,40,64,74
Percentage of first_ndr_to_reattempt,71,72,75,71,70,74,75,69,72,75,72,68,59,12
Percentage of ndr_raised,35,35,47,51,58,54,53,49,46,56,68,65,42,28


In [37]:
# with pd.ExcelWriter('mailsummary.xlsx') as writer:
#     df_all_final.to_excel(writer, sheet_name='Mail')

In [38]:
with pd.ExcelWriter('Smart_Collated_report_latest.xlsx') as writer:
    ranking_summary.to_excel(writer, sheet_name='Ranking_Summary',index=False)
    ranking_cid_final.to_excel(writer, sheet_name='Ranking')
    pickup_summary.to_excel(writer, sheet_name='Pickup_Summary', index=False)
    pickup_cid_final.to_excel(writer, sheet_name='Pickup')
    delivery_summary.to_excel(writer, sheet_name='Delivery_Summary', index=False)
    delivery_cid_final.to_excel(writer, sheet_name='Delivery')
    rto_summary.to_excel(writer, sheet_name='RTO_Summary', index=False)
    rto_prepaid_summary.to_excel(writer, sheet_name='RTO_prepaid_Summary', index=False)
    rto_cod_summary.to_excel(writer, sheet_name='RTO_cod_Summary', index=False)
    rto_cid.to_excel(writer, sheet_name='RTO')
    rto_cid_pre.to_excel(writer, sheet_name='RTO_prepaid_CID')
    rto_cid_cod.to_excel(writer, sheet_name='RTO_cod')  
    ndr_summary.to_excel(writer, sheet_name='NDR_Summary', index=False)
    ndr_pre_summary.to_excel(writer, sheet_name='NDR_prepaid_Summary', index=False)
    ndr_cod_summary.to_excel(writer, sheet_name='NDR_cod_Summary', index=False)
    ndr_cid_final.to_excel(writer, sheet_name='NDR')
    ndr_cid_pre_final.to_excel(writer, sheet_name='NDR_prepaid')
    ndr_cid_cod_final.to_excel(writer, sheet_name='NDR_cod')
    deliv_comp_cod.to_excel(writer, sheet_name='DeliveryPreVsPostSmart_COD')
    deliv_comp_pre.to_excel(writer, sheet_name='DeliveryPreVsPostSmart_prepaid')
    df_all_final.to_excel(writer, sheet_name='MailSummary')
    